In [22]:
import pandas as pd
import numpy as np
import jieba
from sklearn.feature_extraction.text import CountVectorizer

In [60]:
class pLSA:
    def __init__(self,k):
        self.k=k
    def fit(self,X,max_iter=100,random_state=42):
        M,N = X.shape
        N_w = np.sum(X)
        np.random.seed(random_state)

        # Initialize: Dirichlet Distribution => sum = 1
        PwZ = np.random.dirichlet(np.ones(self.k),size=M) # P(w|z)
        PdZ = np.random.dirichlet(np.ones(self.k),size=N) # P(d|z)
        Pz = np.random.dirichlet(np.ones(self.k)) # P(z)

        for iter in range(max_iter):
            # E step
            PzWD = np.einsum('ik,jk,k->kij',PwZ,PdZ,Pz)  # P(z|w,d) !start index:k
            PzWD = PzWD/PzWD.sum(axis=0)

            # M step
            R = np.einsum('ij,kij->ijk',X,PzWD)
            temp = np.einsum('ijk->k',R)
            Pz = temp /N_w
            PwZ = R.sum(axis=1)/temp
            PdZ = R.sum(axis=0)/temp
    
        self.PwZ=PwZ
        self.PdZ=PdZ
        self.Pz=Pz

In [17]:
news = pd.read_csv('../text/News.csv')
display(news.head(3))
category = 'content'
text = news[[category]]

,_id,content,date,source,title
0,197103,6月1日下午，西安交通大学学习贯彻习近平新时代中国特色社会主义思想主题教育巡回指导组对前沿院...,2023-06-02 21:20,交大新闻网,【主题教育】学校主题教育巡回指导组对前沿院党委开展督促指导
1,197096,6月2日，西安交大召开非学历教育工作专题会，校党委常委、副校长柴渭主持会议，校长助理黄忠德出...,2023-06-02 21:32,交大新闻网,西安交大召开非学历教育工作专题会，研究推动非学历教育高质量发展
2,197098,为加强学院新闻宣传队伍建设、提高学院宣传工作能力与水平，6月2日下午，西安交大管理学院党委特...,2023-06-02 21:36,交大新闻网,管理学院举办宣传工作业务能力提升专题培训


In [20]:
with open('hit_stopwords.txt','r') as f: # Import Stopwords
    stopwords=f.read().splitlines()
    print(stopwords[:10])
text['words']=text['content'].apply( # Split and Filter
    lambda line:[x for x in jieba.cut(line) if x not in stopwords]
)
text['words_str']=text['words'].apply(' '.join)
display(text['words_str'].head())

['———', '》），', '）÷（１－', '”，', '）、', '＝（', ':', '→', '℃ ', '&']


C:\Users\19317\AppData\Local\Temp\ipykernel_4292\3015149519.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text['words']=text['content'].apply( # Split and Filter
C:\Users\19317\AppData\Local\Temp\ipykernel_4292\3015149519.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text['words_str']=text['words'].apply(' '.join)


0    6 月 1 日 下午 西安交通大学 学习 贯彻 习近平 新 时代 中国 特色 社会主义 思想...
1    6 月 2 日 西安交大 召开 非学历 教育 工作 专题 会 校党委 常委 副校长 柴渭 主...
2    加强 学院 新闻宣传 队伍 建设 提高 学院 宣传 工作 能力 水平 6 月 2 日 下午 ...
3    5 月 31 日 创中 组织 下 西安交大 机械 学院 党委 副 书记 张敏 创 中心 郭瑞...
4    2023 年 5 月 30 日 佛山市 顺德区 民政 人力资源 社会 保障局 副局长 钟旭辉...
Name: words_str, dtype: object

In [61]:
countvec = CountVectorizer(min_df = 2)
bow=countvec.fit_transform(text['words_str'])
word_list = countvec.get_feature_names()
X=bow.todense().T
print(bow.shape,bow.todense(),countvec.vocabulary_,sep='\n'+'='*20+'\n')

(116, 3228)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
{'下午': 180, '西安交通大学': 2762, '学习': 1216, '贯彻': 2885, '习近平': 327, '时代': 1901, '中国': 257, '特色': 2210, '社会主义': 2368, '思想': 1566, '主题': 309, '教育': 1808, '前沿': 766, '党委': 567, '开展': 1489, '情况': 1587, '进行': 2989, '督促': 2326, '指导': 1701, '校党委': 2017, '常务副': 1428, '书记': 330, '荣命': 2697, '领导班子': 3171, '成员': 1608, '党支部': 573, '参会': 899, '会议': 454, '党委书记': 568, '刘莹': 731, '主持': 297, '汇报': 2103, '一是': 129, '以学': 426, '铸魂': 3069, '根本': 2039, '坚持': 1084, '落实': 2715, '第一': 2456, '议题': 2809, '制度': 754, '学深悟': 1238, '把握': 1663, '理论': 2243, '充分利用': 539, '红色': 2520, '教育资源': 1813, '丰富': 277, '形式': 1529, '迅速': 2954, '二是': 348, '调查': 2860, '研究': 2345, '破解': 2355, '难题': 3125, '显微镜': 1917, '聚焦': 2631, '学院': 1248, '发展': 920, '改革': 1772, '师生': 1413, '领题': 3174, '调研': 2861, '解决': 2789, '急难': 1575, '三是': 162, '促干': 504, '助推': 813, '体制': 471, '机制': 1979, '人才': 398, '引领': 1503, '打造': 16

c:\Users\19317\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [64]:
limit = 10
for K in range(2,6):
    model = pLSA(K)
    model.fit(X)
    print(f'K={K}')
    for k in range(K):
        print(f'Topic:{k+1}')
        freq_words = [word_list[i] for i in model.PwZ[:,k].argsort()[:limit]] 
        print(freq_words)
    print('='*30)


K=2
Topic:1
['推出', '商品房', '良好基础', '新任', '转变职能', '在京举行', '周边', '依靠群众', '紧迫', '纠正']
Topic:2
['成进', '生力军', '氢赋', '肩负着', '营商', '肩负', '传人', '我会', '低碳', '愿景']
K=3
Topic:1
['校地', '框架', '公益', '难以实现', '全球化', '氢赋', '活动中心', '树人', '深厚', '光辉']
Topic:2
['龙建纲', '胜任', '胡锦涛', '胸怀', '创造性地', '刘永忠', '刘文凤', '分组讨论', '分布式', '至今']
Topic:3
['主管部门', '新疆生产建设兵团', '突出重点', '分赴', '刘延东', '创造性地', '力行', '无线电', '督导', '日益']
K=4
Topic:1
['研究员', '回应', '运动竞赛', '回国', '关键技术', '不断创新', '四大', '嘉宾', '输电', '辐射']
Topic:2
['造就', '英才', '高度肯定', '荣命', '荣命哲', '研修', '南京', '产品设计', '产出', '产业链']
Topic:3
['科学馆', '精品课程', '精力', '篮球联赛', '填补', '增加', '增长', '人们', '处分', '管理人员']
Topic:4
['龙建纲', '维持', '两位', '宿舍', '劳动', '努力提高', '助推', '小组会议', '议题', '动员群众']
K=5
Topic:1
['金禾', '选树', '以此', '矛盾', '矢志', '最为', '价值观', '曾经', '思维', '优先发展']
Topic:2
['龙建纲', '协助', '电力', '申请', '由此', '用房', '博士学位', '生动', '生产', '瓶颈']
Topic:3
['龙建纲', '学子', '学术会议', '学术期刊', '学部', '逻辑', '造就', '安徽', '完备', '宏伟蓝图']
Topic:4
['中肯', '新路', '新闻媒体', '予以', '新高', '无线电', '无限', '时代背景', '明确要求', '智能化']
